In [10]:
import pandas as pd
import torch
import torch.nn.functional as F
from model import CustomModel
import numpy as np

In [11]:
df = pd.read_csv("dataset/test.csv")
df.drop(["id"], axis=1, inplace=True)
column_list = list(df.columns)
del column_list[-1]
column_list.append("type_int")
df_type = df["type"]
df = (df-df.mean())/df.std()
df["type_int"] = df_type.map(lambda x : -1 if x == "white" else 1)
df = df[column_list]
best_epoch_fold_0 = 64
best_epoch_fold_1 = 74
best_epoch_fold_2 = 4
best_epoch_fold_3 = 39
best_epoch_fold_4 = 29

In [12]:
test_dataset = df.values

In [13]:
dataset = torch.from_numpy(test_dataset).float()

In [14]:
checkpoint_0 = torch.load(f"fold0/model_{best_epoch_fold_0}.pt")
checkpoint_1 = torch.load(f"fold1/model_{best_epoch_fold_1}.pt")
checkpoint_2 = torch.load(f"fold2/model_{best_epoch_fold_2}.pt")
checkpoint_3 = torch.load(f"fold3/model_{best_epoch_fold_3}.pt")
checkpoint_4 = torch.load(f"fold4/model_{best_epoch_fold_4}.pt")

In [15]:
model_0 = CustomModel(12, 5)
model_0.load_state_dict(checkpoint_0["model_state_dict"])
model_1 = CustomModel(12, 5)
model_1.load_state_dict(checkpoint_1["model_state_dict"])
model_2 = CustomModel(12, 5)
model_2.load_state_dict(checkpoint_2["model_state_dict"])
model_3 = CustomModel(12, 5)
model_3.load_state_dict(checkpoint_3["model_state_dict"])
model_4 = CustomModel(12, 5)
model_4.load_state_dict(checkpoint_4["model_state_dict"])

<All keys matched successfully>

In [19]:
preds_0 = F.softmax(model_0(dataset), dim=-1)
preds_1 = F.softmax(model_1(dataset), dim=-1)
preds_2 = F.softmax(model_2(dataset), dim=-1)
preds_3 = F.softmax(model_3(dataset), dim=-1)
preds_4 = F.softmax(model_4(dataset), dim=-1)

In [22]:
preds = preds_0 + preds_1 + preds_2 + preds_3 + preds_4

In [25]:
res = torch.argmax(preds, axis=-1)

In [27]:
res_np = res.numpy()
res_np = res_np + 4

In [28]:
idx = np.array(range(1, res_np.shape[0]+1))

In [29]:
df = pd.DataFrame(zip(idx, res_np), columns=["id", "quality"])

In [30]:
df.to_csv("result.csv", index=False)